Pour faire tourner les cellules, il vous faut:

- Télécharger le fichier .txt présent sur <a href="https://wikipedia2vec.github.io/wikipedia2vec/pretrained/">cette page</a>. <br>
  Il s'agit du modèle pré-entrainé qui implémente word2vec ie il convertit chaque mot qu'on lui donne en un vecteur de taille p.<br>
  La taille p du vecteur peut être choisie sur cette page de téléchargement (il y a, en français, le choix entre 100 et 300).<br>
  Pour le télécharger, cliquer sur le ".txt" n'a pas suffit pour moi, il glisser le lien dans un nouvel onglet. <br>
  Le chargement (sur mon ordinateur) de la version avec p=100 prend environ 5mins et celle avec p=300 prend bien 20 mins.<br>
  Il faut compter quelques Gb en RAM.

- Télécharger les modules présent dans la cellule en dessous.

In [ ]:
#Importation des modules nécessaires
#En raison de la taille du modèle utilisé, il est nécessaire d'utiliser Google Colab pour exécuter ce code

import string
import spacy
from gensim.models import keyedvectors
import numpy as np
from google.colab import drive
import pandas as pd
import requests
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from googletrans import Translator
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#à exécuter la première fois
!pip install --upgrade googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=cb11603ea5999764005687bea899e5617a800bdf87bddfa90394a51b264bff44
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

In [ ]:
CATEGORIES = ["politique",
                "environnement",
                "agriculture",
                "économie",
                "éducation",
                "transport",
                "religion",
                "santé",
                "travail",
                "sport",
                "justice",
                "loisir",
                "social",
                "technologie",
                "art"]

In [ ]:
class Word2Vec:
    def __init__(self, path_model, categories):
        self.path_model = path_model
        self.categories = categories
        self.nlp = self.load_spacy_model()
        self.trained = None

    @staticmethod
    def load_spacy_model():
        try:
            return spacy.load("fr_core_news_sm")
        except OSError:
            spacy.cli.download("fr_core_news_sm")
            return spacy.load("fr_core_news_sm")

    def train(self):
        #Cette étape prend du temps. Il faut compter environ 10 minutes pour charger le modèle en dimension 100
        #Il faut plutôt 20mins pour charger le modèle en dimension 300. Ces temps sont pris sur Google Colab.
        self.trained = keyedvectors.load_word2vec_format(self.path_model, binary=False)

    def _update_vector(self, vec, token):
        try:
            values = self.trained[token]
            vec += values
            return vec, True
        except KeyError:
            return vec, False

    def my_doc_2_vec(self, mots):
        #Permet à partir d'une liste de mot, de retourner un vecteur moyen.
        #Il s'agit du barycentre des vecteurs des mots du texte.
        vec = np.zeros(self.trained.vectors.shape[1])
        count = 0
        for token in mots:
            vec, updated = self._update_vector(vec, token)
            if updated:
                count += 1
        return vec/count if count > 0 else vec

    def _lemmatize_and_filter(self, texte):
        #Permet de lemmatiser et de filtrer les mots du texte.
        doc = self.nlp(texte)
        lemmes = [token.lemma_ for token in doc]
        return [
            lemme for lemme in lemmes
            if lemme not in self.nlp.Defaults.stop_words and lemme not in string.punctuation
        ]

    def get_vect(self, texte):
        #Permet de retourner le vecteur moyen d'un texte (comprend la lematisation, le filtrage des mots et le calcul du barycentre)
        mots_filtrés = self._lemmatize_and_filter(texte)
        return self.my_doc_2_vec(mots_filtrés)

    @staticmethod
    def dist(word_vector1, word_vector2):
        #Permet de calculer la similarité cosinus entre deux vecteurs
        return np.dot(word_vector1, word_vector2) / (np.linalg.norm(word_vector1) * np.linalg.norm(word_vector2))

    def get_sorted_cats(self, texte):
        #Permet de retourner les catégories les plus proches d'un texte dans l'ordre
        vec = self.get_vect(texte)
        cat_vectors = [self.trained[cat] for cat in self.categories]
        distances = [self.dist(cat_vec, vec) for cat_vec in cat_vectors]
        sorted_categories = sorted(zip(self.categories, distances), key=lambda x: -x[1])
        return sorted_categories


In [ ]:
class Sentiments:
  def __init__(self):
    try:
      self.analyzer = SentimentIntensityAnalyzer()
    except:
      nltk.download('vader_lexicon')
      self.analyzer = SentimentIntensityAnalyzer()
    self.translator = Translator()

  def get_sentiment_analysis(self,phrase):
    #Permet de retourner la polarité d'une phrase
    #On traduit la phrase en anglais pour une meilleure analyse
    phrase_en = self.translator.translate(phrase, dest='en').text
    return self.analyzer.polarity_scores(phrase_en)["compound"]

In [ ]:
my_sentiments_model = Sentiments()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
my_word2vec_model = Word2Vec(r"/content/gdrive/MyDrive/Colab Notebooks/frwiki_20180420_300d.txt.bz2",CATEGORIES)
#Il faut bien sûr changer le chemin du modèle pour qu'il corresponde à l'emplacement du modèle sur votre Google Drive

In [ ]:
my_word2vec_model.train()

In [ ]:
NOTION_API_URL = "https://api.notion.com/v1/pages"
NOTION_API_KEY = "INSERER API KEY"
DATABASE_ID_REP_TRAITE = "6fb0749269924363afa68bd3fab90fa2"
DATABASE_ID_REP_BRUTE = "d6e83d295d2c40548fdc0fa0241a24c4"
DATABASE_ID_REP_BRUTE2 = "16ea2b5b9b7547a3883b6202160ec2e7"

def add_rep_traite(data):
  #Permet d'ajouter une réponse traitée à la base de données Notion
  headers = {
      "Authorization": f"Bearer {NOTION_API_KEY}",
      "Content-Type": "application/json",
      "Notion-Version": "2022-06-28"
  }

  payload = {
      "parent": {"database_id": DATABASE_ID_REP_TRAITE},
      "properties": {
          k: {"rich_text": [{"text": {"content": str(data[k])}}]} for k in data.keys()
      }
  }

  response = requests.post(NOTION_API_URL, json=payload, headers=headers)
  if response.status_code == 200:
    pass
  else:
      print(f"Failed to add entry to Notion. Status code: {response.status_code}, Response: {response.text}")

In [ ]:
def get_data_from_notion(DATABASE_ID):
    #Permet de récupérer les données d'une base de données Notion à partir de son ID
    NOTION_QUERY_URL = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"
    headers = {
        "Authorization": f"Bearer {NOTION_API_KEY}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"
    }

    rows = {}
    has_more = True
    start_cursor = None

    while has_more:
        payload = {"start_cursor": start_cursor} if start_cursor else {}
        response = requests.post(NOTION_QUERY_URL, headers=headers, json=payload)
        data = response.json()

        if not rows:  # Initialize columns on the first request
            column_names = data["results"][0]['properties'].keys()
            rows = {col: [] for col in column_names if col != "empty_col"}

        for result in data['results']:
            properties = result['properties']
            for k in rows.keys():
                if 'rich_text' in properties[k] and properties[k]['rich_text']:
                    rows[k].append(properties[k]['rich_text'][0]['text']['content'])
                else:
                    rows[k].append(None)

        has_more = data.get("has_more", False)
        start_cursor = data.get("next_cursor", None)

    return rows

In [ ]:
def get_new_data():
  #Permet de récupérer les nouvelles réponses brutes qui n'ont pas encore été traitées
  #Pour cela, nous avons mis en place un système d'indexage qui à une réponse associe un unique identifiant.
  #Nous comparons les identifiants des réponses brutes et des réponses traitées pour déterminer les nouvelles réponses.
  data_brute = get_data_from_notion(DATABASE_ID_REP_BRUTE2)
  data_traite = get_data_from_notion(DATABASE_ID_REP_TRAITE)
  new_data = {"Rep1":[],
            "Rep2":[],
            "Rep3":[],
            "Index":[]}
  for i in range(len(data_brute["Index"])):
    if not (data_brute["Index"][i] in data_traite["num"]):
      for k in new_data.keys():
        new_data[k].append(data_brute[k][i])
  return new_data

In [ ]:
def traiter_data(new_data,word2vec_model,sentiment_model):
  #Permet de traiter les nouvelles réponses brutes (ie les catégoriser et analyser leur sentiment).
  df = pd.DataFrame(new_data)
  df["phrase"] = "En tant que " + df["Rep1"] + " j'aimerais " + df["Rep2"] + " parce que " + df["Rep3"]
  df2 = pd.concat((df["Index"],df["phrase"],df["phrase"].apply(word2vec_model.get_sorted_cats)),axis=1)
  df2["emotion"] = df["phrase"].apply(sentiment_model.get_sentiment_analysis)
  cats = {cat : [] for cat in CATEGORIES}
  cats["num"] = []
  cats["emotion"] = []

  for i in range(len(df2)):
    s = df2.iloc[i]
    for j in range(len(cats.keys())-2):
      cats[s.iloc[2][j][0]].append(s.iloc[2][j][1])
    cats["num"].append(int(s["Index"]))
    cats["emotion"].append(s["emotion"])
  return cats

In [ ]:
def update_rep_traite(word2vec_model,sentiment_model):
  #Permet de mettre à jour la base de données Notion avec les réponses traitées
  new_data = get_new_data()
  if len(new_data["Index"]) == 0:
    print("Nothing to update !")
    return False
  new_data_traite = traiter_data(new_data,word2vec_model,sentiment_model)

  for i in range(len(new_data_traite["num"])):
    new_sample = {cat : [] for cat in CATEGORIES}
    new_sample["num"] = []
    for k in new_data_traite.keys():
      new_sample[k] = new_data_traite[k][i]
    add_rep_traite(new_sample)
    print(f"Added {i+1}/{len(new_data_traite['num'])} samples.",end="\r")

In [ ]:
update_rep_traite(my_word2vec_model,my_sentiments_model)

4
{'Rep1': ['Personnels des services directs aux particuliers', 'Personnels des services directs aux particuliers', 'Employés de commerce', 'Employés de commerce', 'Employés de commerce', 'Employés de commerce', 'Employés de commerce', 'Employés de commerce', 'Employés de commerce', 'Employés de commerce', 'Employés de commerce', 'Employés de commerce', "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Employés administratifs d'entreprise", "Emp